# base_line 모델(roberta-base)의 선정 및 f1_score & pearsonr

Team LostCow의 roberta모델을 기준으로 함

출처: https://github.com/l-yohai/KLUE

pretrained_model : klue/roberta-base()

tokenizer : klue/roberta-base

dropout : 0.1

batch_size : 64

optim /learning_rate : adamw_hf / 5e-5

loss_fnc : MSELoss

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 7.3 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.5 MB 30.4 MB/s 
     |████████████████████████████████| 895 kB 35.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 6.4 MB/s 
     |████████████████████████████████| 212 kB 19.4 MB/s 
     |████████████████████████████████| 134 kB 22.8 MB/s 
     |████████████████████████████████| 1.1 MB 21.0 MB/s 
     |████████████████████████████████| 127 kB 50.8 MB/s 
     |████████████████████████████████| 144 kB 34.0 MB/s 
     |████████████████████████████████| 271 kB 51.2 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
import tarfile

#훈련 데이터셋 다운로드 및 압축풀기

!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

tar_bz2_file = tarfile.open("/content/klue-sts-v1.1.tar.gz")
tar_bz2_file.extractall(path="/content")
tar_bz2_file.close()

--2022-03-17 07:12:50--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz
Resolving aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)... 52.218.169.187
Connecting to aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)|52.218.169.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1349881 (1.3M) [application/x-gzip]
Saving to: ‘klue-sts-v1.1.tar.gz’

klue-sts-v1.1.tar.g 100%[===================>]   1.29M  --.-KB/s    in 0.08s   

2022-03-17 07:12:50 (16.4 MB/s) - ‘klue-sts-v1.1.tar.gz’ saved [1349881/1349881]



In [ ]:
import torch

#GPU 사용 여부 확인 및 name 확인

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [3]:
import json

#train 데이터셋을 train,valid 데이터셋으로 split

def read_json(file_path):
    with open(file_path) as f:
        return json.load(f)

data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_train.json')

train_length=int(len(data)*0.9)
train=data[:train_length]
vaild=data[train_length:]

print('data_set:',len(data))
print('train_set:',len(train),', valid_set:',len(vaild))

with open('train_split.json','w') as f:
  json.dump(train,f,ensure_ascii = False)
with open('valid_split.json','w') as f:
  json.dump(vaild,f,ensure_ascii = False)

data_set: 11668
train_set: 10501 , valid_set: 1167


In [ ]:
!git clone https://github.com/l-yohai/KLUE #base line 모델 불러오기

Cloning into 'KLUE'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 313 (delta 52), reused 101 (delta 29), pack-reused 179
Receiving objects: 100% (313/313), 5.79 MiB | 13.87 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [ ]:
# base line 모델 학습
!python /content/KLUE/sts/train.py --data_dir /content --model_name_or_path "klue/roberta-base" --train_filename "train_split.json" --valid_filename "valid_split.json" --num_train_epochs 10 --save_steps 100 --save_steps 100

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForStsRegression: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForStsRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForStsRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForStsRegression were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['cls_fc_layer.linear.weight', 'sentence_fc_layer.linear.bias', 'dense.linear.weight', 'roberta.pool

In [ ]:
mv /content/klue-sts-v1.1.tar.gz /content/model/klue-sts-v1.1.tar.gz

In [ ]:
#학습한 base line 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/KLUE/sts/inference.py --test_filename "/content/klue-sts-v1.1/klue-sts-v1.1_dev.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [ ]:
#valid의 label 값을 추출

valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]

In [ ]:
from sklearn.metrics import classification_report,f1_score
import pandas as pd

#Regression pred 를 binary-label pred로 변환 및 혼돈 메트릭스 및 f1_score 시각화

df_pred = pd.read_csv('/content/output.csv',header=None)

pred=(df_pred>2.5).astype(int)
print("epochs 10 / >2.5")
print(classification_report(valid_label, pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, pred),end='\n\n')

pred=(df_pred>3.1).astype(int)
print("epochs 10 / >3.1")
print(classification_report(valid_label, pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, pred))

epochs 10 / >2.5
              precision    recall  f1-score   support

           0       1.00      0.66      0.79       299
           1       0.68      1.00      0.81       220

    accuracy                           0.80       519
   macro avg       0.84      0.83      0.80       519
weighted avg       0.86      0.80      0.80       519

base_line(roberta-base) f1_score: 0.8103130755064457

epochs 10 / >3.1
              precision    recall  f1-score   support

           0       0.96      0.80      0.87       299
           1       0.78      0.96      0.86       220

    accuracy                           0.87       519
   macro avg       0.87      0.88      0.87       519
weighted avg       0.89      0.87      0.87       519

base_line(roberta-base) f1_score: 0.859470468431772


In [ ]:
from datasets import load_metric

#dev_set에 대한 pearsonr score 출력 

df_pred = pd.read_csv('/content/output.csv',header=None)
valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['real-label'] for data in valid_data]

pearson = load_metric("pearsonr").compute
metric = pearson(predictions=df_pred.to_numpy(), references=valid_label)

print(metric)

{'pearsonr': 0.8939478658048167}


# 모델(roberta-base)의 custom 및 f1_score & pearsonr

모델의 FClayer 층이 참고하는 roberta model의 hidden states를 다각화 하여 성능을 향상시켰습니다.

base model로 선정한 LostCow team의 FClayer층의 학습 방식은 roberta model이 문장A,B에 대해서 token_type_ids 값을 입력하지 않고 구분되지 않은 상태로 학습시키며 마지막 layer의 임베딩벡터를 문장A,B의 위치를 마스킹해둔 s1_mask, s2_mask를 이용하여 분리한뒤 각 단어들의 dim의 위치값을 평균내어 하나의 토큰 임베딩 값으로 생성한 후 FClayer를 통해 연산됩니다.

이 때문에 roberta model의 pooled_output을 사용하지 않으며 사용시에도 성능 향상에 기여하지 못합니다.

각 문장의 임베딩 벡터의 평균값을 기준으로 FClayer의 연산이 이루어지는 점을 이용하여 각 문장의 의미가 적당하게 고려된 hidden_states layer를 탐색하여 2,3layer의 hidden_states를 추가로 concat하는 것으로 모델의 성능을 향상시켰습니다. 

In [ ]:
"""   

  -------------------base model(model.forward code)----------------------
        outputs = self.roberta(
            input_ids, attention_mask=attention_mask, token_type_ids=None
        )
        sequence_output = outputs[0]
        pooled_output = outputs[1]
        s1_h = self.entity_average(sequence_output, s1_mask)
        s2_h = self.entity_average(sequence_output, s2_mask)
        s1_h = self.sentence_fc_layer(s1_h)
        s2_h = self.sentence_fc_layer(s2_h)

        concat_h = torch.cat([s1_h, s2_h], dim=-1)
        concat_h = self.dense(concat_h)

        logits = self.label_classifier(concat_h)

        outputs = (logits,) + outputs[2:]




      ----------------custom model(model.forward code)---------------------

        outputs = self.roberta(
            input_ids, attention_mask=attention_mask, token_type_ids=None, output_hidden_states=True
        )

        sequence_output = outputs[0]
        pooled_output = outputs[1]
        s1_h = self.entity_average(sequence_output, s1_mask)
        s2_h = self.entity_average(sequence_output, s2_mask)
        s1_h = self.sentence_fc_layer(s1_h)
        s2_h = self.sentence_fc_layer(s2_h)
        
        sequence_output_sec_layer=outputs['hidden_states'][2]
        ss1_h = self.entity_average(sequence_output_sec_layer, s1_mask)
        ss2_h = self.entity_average(sequence_output_sec_layer, s2_mask)
        ss1_h = self.sentence_fc_layer2(ss1_h)
        ss2_h = self.sentence_fc_layer2(ss2_h)

        sequence_output_sec_layer2=outputs['hidden_states'][3]
        sss1_h = self.entity_average(sequence_output_sec_layer2, s1_mask)
        sss2_h = self.entity_average(sequence_output_sec_layer2, s2_mask)
        sss1_h = self.sentence_fc_layer3(sss1_h)
        sss2_h = self.sentence_fc_layer3(sss2_h)

        concat_h = torch.cat([s1_h, s2_h,ss1_h, ss2_h,sss1_h, sss2_h], dim=-1)
        concat_h = self.dense(concat_h)
        concat_h = self.dense2(concat_h)
        logits = self.label_classifier(concat_h)

        outputs = (logits,) + ()

"""

In [4]:
#모델 custom을 진행한 폴더 압축해제
!unzip /content/KLUE_custom.zip

Archive:  /content/KLUE_custom.zip
  inflating: sts/__pycache__/dataset.cpython-37.pyc  
  inflating: sts/__pycache__/metric.cpython-37.pyc  
  inflating: sts/__pycache__/model.cpython-37.pyc  
  inflating: sts/__pycache__/utils.cpython-37.pyc  
  inflating: sts/data/download.sh    
  inflating: sts/README.md           
  inflating: sts/dataloader.py       
  inflating: sts/dataset.py          
  inflating: sts/infer_test.ipynb    
  inflating: sts/inference.py        
  inflating: sts/metric.py           
  inflating: sts/model.py            
 extracting: sts/requirements.txt    
  inflating: sts/train.py            
  inflating: sts/utils.py            


In [5]:
# custom 모델 학습
!python /content/sts/train.py --data_dir /content --model_name_or_path "klue/roberta-base" --train_filename "train_split.json" --valid_filename "valid_split.json" --num_train_epochs 10 --save_steps 100 --save_steps 100

Downloading: 100% 546/546 [00:00<00:00, 502kB/s]
Downloading: 100% 375/375 [00:00<00:00, 345kB/s]
Downloading: 100% 243k/243k [00:00<00:00, 994kB/s]
Downloading: 100% 734k/734k [00:00<00:00, 2.34MB/s]
Downloading: 100% 173/173 [00:00<00:00, 158kB/s]
Downloading: 100% 422M/422M [00:10<00:00, 42.7MB/s]
Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForStsRegression: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForStsRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForStsRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenc

In [18]:
#학습한 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/sts/inference.py --test_filename "/content/klue-sts-v1.1/klue-sts-v1.1_dev.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [19]:
from sklearn.metrics import classification_report,f1_score
import pandas as pd


#valid의 label 값을 추출

valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]


#Regression pred 를 binary-label pred로 변환 및 혼돈 메트릭스 및 f1_score 시각화

df_pred = pd.read_csv('/content/output.csv',header=None)

pred=(df_pred>2.5).astype(int)
print("epochs 10 / >2.5")
print(classification_report(valid_label, pred))
print('custom_model(roberta-base) f1_score:',f1_score(valid_label, pred),end='\n\n')

pred=(df_pred>2.85).astype(int)
print("epochs 10 / >2.85")
print(classification_report(valid_label, pred))
print('custom_model(roberta-base) f1_score:',f1_score(valid_label, pred))

epochs 10 / >2.5
              precision    recall  f1-score   support

           0       0.99      0.71      0.83       299
           1       0.71      0.99      0.83       220

    accuracy                           0.83       519
   macro avg       0.85      0.85      0.83       519
weighted avg       0.87      0.83      0.83       519

custom_model(roberta-base) f1_score: 0.8304761904761905

epochs 10 / >2.85
              precision    recall  f1-score   support

           0       0.97      0.80      0.88       299
           1       0.78      0.97      0.86       220

    accuracy                           0.87       519
   macro avg       0.88      0.88      0.87       519
weighted avg       0.89      0.87      0.87       519

custom_model(roberta-base) f1_score: 0.8640973630831643


In [20]:
from datasets import load_metric


#dev_set에 대한 pearsonr score 출력 

df_pred = pd.read_csv('/content/output.csv',header=None)
valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['real-label'] for data in valid_data]

pearson = load_metric("pearsonr").compute
metric = pearson(predictions=df_pred.to_numpy(), references=valid_label)
print(metric)

{'pearsonr': 0.8981787692339827}


# 학습 데이터의 전처리에 따른 성능 비교

전처리 1 : 영어, 숫자, 특수문자 전처리

전처리 2 : 영어만 소문자 처리

다음의 두가지 방식으로 전처리 진행후 성능을 비교하였습니다.

결과 전처리를 통한 성능 감소를 확인 하였고 전처리를 진행하지 않고 학습시킨 custom 모델을 최종 선정하였습니다.

In [28]:
import copy

# 파일 업로드

with open('/content/klue-sts-v1.1/klue-sts-v1.1_train.json', 'r') as f:

  json_data_1 = json.load(f)
  json_data_2 = copy.deepcopy(json_data_1)

  json.dumps(json_data_1, ensure_ascii=False)
  json.dumps(json_data_2, ensure_ascii=False)

with open('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json', 'r') as f:

  json_data_dev_1 = json.load(f)
  json_data_dev_2 = copy.deepcopy(json_data_dev_1)

  json.dumps(json_data_dev_1, ensure_ascii=False)
  json.dumps(json_data_dev_2, ensure_ascii=False)

In [22]:
import re

# 전처리 두가지 1. 영어, 숫자, 특수문자 전처리 2. 영어만 소문자 처리

# 1번 전처리(영어, 숫자, 특수문자 전처리)
def delete_1(json_data):
  for idx in range(len(json_data)):
    json_data[idx]['sentence1'] = re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', json_data[idx]['sentence1'])
    json_data[idx]['sentence2'] = re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', json_data[idx]['sentence2'])
  return json_data

# 2번 전처리(영어만 소문자 처리)
def delete_2(json_data):
  for idx in range(len(json_data)):
    json_data[idx]['sentence1'] = json_data[idx]['sentence1'].lower()
    json_data[idx]['sentence2'] = json_data[idx]['sentence2'].lower()
  return json_data


In [33]:
#전처리 진행 

train_1 = delete_1(json_data_1)
train_2 = delete_2(json_data_2)
dev_1 = delete_1(json_data_dev_1)
dev_2 = delete_1(json_data_dev_2)

# 전처리 결과 확인

print("전처리 1 :",train_1[7162]['sentence1'])
print("전처리 2 :",train_2[7162]['sentence1'])

전처리 1 : 올해 예산 이·전용분 억 원 중 억 원은 코박스 퍼실러티  가입을 위한 선급금으로 이미 집행했다
전처리 2 : 올해 예산 이·전용분 1723억 원 중 850억 원은 코박스 퍼실러티(covax facility) 가입을 위한 선급금으로 이미 집행했다.


In [37]:
# 전처리 파일 json파일 저장

with open('/content/train_1.json','w') as f:
  json.dump(train_1, f, ensure_ascii=False)

with open('/content/train_2.json','w') as f:
  json.dump(train_2, f, ensure_ascii=False)

with open('/content/dev_1.json','w') as f:
  json.dump(dev_1, f, ensure_ascii=False)

with open('/content/dev_2.json','w') as f:
  json.dump(dev_2, f, ensure_ascii=False)

In [26]:
#train 데이터셋을 train,valid 데이터셋으로 split


def read_json(file_path):
    with open(file_path) as f:
        return json.load(f)

for num in range(1,3):
  data=read_json(f'/content/train_{num}.json')

  train_length=int(len(data)*0.9)
  train=data[:train_length]
  vaild=data[train_length:]

  print('data_set:',len(data))
  print('train_set:',len(train),', valid_set:',len(vaild))

  with open(f'train_split{num}.json','w') as f:
    json.dump(train,f,ensure_ascii = False)
  with open(f'valid_split{num}.json','w') as f:
    json.dump(vaild,f,ensure_ascii = False)

data_set: 11668
train_set: 10501 , valid_set: 1167
data_set: 11668
train_set: 10501 , valid_set: 1167


## 전처리 1에 따른 모델 학습 및 성능 비교

In [31]:
# 1번 전처리 데이터로 custom 모델 학습
!python /content/sts/train.py --data_dir /content --model_name_or_path "klue/roberta-base" --train_filename "train_split1.json" --valid_filename "valid_split1.json" --num_train_epochs 10 --save_steps 100 --save_steps 100

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForStsRegression: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForStsRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForStsRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForStsRegression were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'cls_fc_layer.linear.bias', 'dense2.linear.weight', 'cls_fc_layer.line

In [38]:
#학습한 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/sts/inference.py --test_filename "/content/dev_1.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [48]:
#valid의 label 값을 추출

valid_data=read_json('/content/dev_1.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]


#Regression pred 를 binary-label pred로 변환 및 혼돈 메트릭스 및 f1_score 시각화

df_pred = pd.read_csv('/content/output.csv',header=None)

pred=(df_pred>2.5).astype(int)
print("epochs 10 / >2.5")
print(classification_report(valid_label, pred))
print('custom_model(roberta-base / 전처리1) f1_score:',f1_score(valid_label, pred),end='\n\n')

pred=(df_pred>3.2).astype(int)
print("epochs 10 / >3.2")
print(classification_report(valid_label, pred))
print('custom_model(roberta-base / 전처리1) f1_score:',f1_score(valid_label, pred))

epochs 10 / >2.5
              precision    recall  f1-score   support

           0       0.98      0.66      0.79       299
           1       0.68      0.98      0.80       220

    accuracy                           0.80       519
   macro avg       0.83      0.82      0.80       519
weighted avg       0.85      0.80      0.80       519

custom_model(roberta-base / 전처리1) f1_score: 0.8044692737430168

epochs 10 / >3.2
              precision    recall  f1-score   support

           0       0.91      0.82      0.86       299
           1       0.78      0.90      0.83       220

    accuracy                           0.85       519
   macro avg       0.85      0.86      0.85       519
weighted avg       0.86      0.85      0.85       519

custom_model(roberta-base / 전처리1) f1_score: 0.8347457627118645


In [49]:
#dev_set에 대한 pearsonr score 출력 

df_pred = pd.read_csv('/content/output.csv',header=None)
valid_data=read_json('/content/dev_1.json')
valid_label=[data['labels']['real-label'] for data in valid_data]

pearson = load_metric("pearsonr").compute
metric = pearson(predictions=df_pred.to_numpy(), references=valid_label)
print(metric)

{'pearsonr': 0.8468598368217276}


## 전처리 2에 따른 모델 학습 및 성능 비교

In [50]:
# 2번 전처리 데이터로 custom 모델 학습
!python /content/sts/train.py --data_dir /content --model_name_or_path "klue/roberta-base" --train_filename "train_split2.json" --valid_filename "valid_split2.json" --num_train_epochs 10 --save_steps 100 --save_steps 100

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForStsRegression: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForStsRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForStsRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForStsRegression were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'sentence_fc_layer3.linear.weight', 'dense2.linear.bias', 'cls_fc_la

In [51]:
#학습한 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/sts/inference.py --test_filename "/content/dev_2.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [64]:
#valid의 label 값을 추출

valid_data=read_json('/content/dev_2.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]


#Regression pred 를 binary-label pred로 변환 및 혼돈 메트릭스 및 f1_score 시각화

df_pred = pd.read_csv('/content/output.csv',header=None)

pred=(df_pred>2.5).astype(int)
print("epochs 10 / >2.5")
print(classification_report(valid_label, pred))
print('custom_model(roberta-base / 전처리2) f1_score:',f1_score(valid_label, pred),end='\n\n')

pred=(df_pred>2.8).astype(int)
print("epochs 10 / >2.8")
print(classification_report(valid_label, pred))
print('custom_model(roberta-base / 전처리2) f1_score:',f1_score(valid_label, pred))

epochs 10 / >2.5
              precision    recall  f1-score   support

           0       0.96      0.70      0.81       299
           1       0.70      0.96      0.81       220

    accuracy                           0.81       519
   macro avg       0.83      0.83      0.81       519
weighted avg       0.85      0.81      0.81       519

custom_model(roberta-base / 전처리2) f1_score: 0.8084291187739464

epochs 10 / >2.8
              precision    recall  f1-score   support

           0       0.93      0.77      0.84       299
           1       0.75      0.92      0.82       220

    accuracy                           0.83       519
   macro avg       0.84      0.84      0.83       519
weighted avg       0.85      0.83      0.83       519

custom_model(roberta-base / 전처리2) f1_score: 0.8228105906313646


In [65]:
#dev_set에 대한 pearsonr score 출력 

df_pred = pd.read_csv('/content/output.csv',header=None)
valid_data=read_json('/content/dev_2.json')
valid_label=[data['labels']['real-label'] for data in valid_data]

pearson = load_metric("pearsonr").compute
metric = pearson(predictions=df_pred.to_numpy(), references=valid_label)
print(metric)

{'pearsonr': 0.8503741504839647}
